In [1]:
# 1. Load Domain
from dialog_agents import RestaurantAgent
from dialog_simulator import *
import yaml

domain_manager = DomainManager()
domain_manager.load_domains("data/")
restaurant = domain_manager.get_domain("restaurant")

# 2. Create User simulator object
# a. Create User Simulator object 
from user_simulator import RuleSimulator
user_sim = RuleSimulator(restaurant, "template")

# b. Load rule based NLG model for User
nlg_dict = yaml.safe_load(open("data/nlg_rules.yaml"))
nlg_model = NLG("dict", nlg_dict)

In [18]:
import random
import uuid
#user_sim.set_nlg(nlg_model)
#user_sim.generate_goal("template")

def setup_usersim(domain, goaltype, starting_goals_path, nlg=None, nlu=None):
    usersim = RuleSimulator(domain, goaltype)
    usersim.set_nlg(nlg)
    usersim.set_nlu(nlu)
    usersim.load_starting_goals(starting_goals_path, "yaml")
    usersim.generate_goal(goaltype)
    
    return usersim

def setup_agent(domain, nlg=None, nlu=None):
    agent = RestaurantAgent(domain)
    agent.set_nlu(nlu)
    agent.set_nlg(nlg)
    return agent

def print_action(action, turn, speaker):
    print(turn,': ', speaker, "->", action.dialog_act, action.params)
    print(speaker, ": ", action.nl_utterance )
    
def take_turn(action, turn, speaker):
    # 1. If speaker has no NLU, directly pass frame
    if speaker.get_nlu() is None:
        action = speaker.next(action,turn)
    # Add in else logic
    # a. pass NL utterance of previous speaker to nlu of current speaker
    # b. pass action frame return by nlu to next()
    return action

def evaluate_dialog(user_goal):
    for k,v in user_goal.request_slots.items():
        if v == "UNK":
            return False
    return True
    
def register_turn(user_action, agent_action, user_goal ,turn, first_speaker):
    if first_speaker == "usersim":
        conv_round = { "turn": turn,
                       "user_goal": user_goal,
                       "first_speaker": {
                           "speaker": "usersim",
                           "dialog_act": user_action.dialog_act,
                           "dialog_slots": user_action.params,
                           "nl_utterance": user_action.nl_utterance
                       },
                       "second_speaker": {
                           "speaker": "agent",
                           "dialog_act":   agent_action.dialog_act,
                           "dialog_slots": agent_action.params,
                           "nl_utterance": agent_action.nl_utterance 
                       }
                     }
    else:
        conv_round = { "turn": turn,
                       "user_goal": user_goal, 
                       "first_speaker": {
                           "speaker": "agent",
                           "dialog_act":   agent_action.dialog_act,
                           "dialog_slots": agent_action.params,
                           "nl_utterance": agent_action.nl_utterance 
                       },
                       "second_speaker": {
                           "speaker": "usersim",
                           "dialog_act": user_action.dialog_act,
                           "dialog_slots": user_action.params,
                           "nl_utterance": user_action.nl_utterance
                       }
                 }
    # Update Dialog History
    dialog_history.append( conv_round )

def register_simulated_dialog(dialog_history, final_user_goal, turns_taken):
    # 1. Evaluate dialog 
    dialog_grade = "Success" if evaluate_dialog(final_user_goal) else "Failed"
    
    # 2. Evaluate if agent suggestion matches user preferences
    goal_grade = "Success"
    
    # 2. register dialog in all_dialogs
    all_dialogs.append({ "dialog_id": str(uuid.uuid4()),
                         "dialog_grade": dialog_grade,
                         "goal_grade": goal_grade,
                         "reward": 0 ,
                         "turns_taken": turns_taken,
                         "turn_history": dialog_history })

##############################################################################################

all_dialogs = []

max_turns = 8
turn = 1

# 1 Create agent and usersim
agent = setup_agent(restaurant)
goal_path = "data/sample_starting_goals.yaml"
user_sim = setup_usersim(domain=restaurant, goaltype="template",
                         starting_goals_path=goal_path , nlg=nlg_model)

print(user_sim.goal.get_goal())

# Stash user goal
user_goal = user_sim.goal

# 2. Flip coin for who goes first
flip = 1

user_action = None
agent_action = None

dialog_history = []
while turn <= max_turns:  
    
    if (user_sim.dialog_status == DialogStatus.FINISHED or 
        agent.dialog_status == DialogStatus.FINISHED):
        break
        
    if flip == 0:
        user_action = take_turn(agent_action, turn, user_sim)
        print_action(user_action, turn, "User")
        agent_action = take_turn(user_action, turn, agent)
        register_turn(user_action, agent_action, user_sim.goal.get_goal(), turn, "usersim")
        print_action(agent_action, turn, "Agent")
    else:
        agent_action = take_turn(user_action, turn, agent)
        print_action(agent_action, turn, "Agent")
        user_action = take_turn(agent_action, turn, user_sim)
        print_action(user_action, turn, "User")
        register_turn(user_action, agent_action,user_sim.goal.get_goal(), turn, "usersim")
    turn += 1
        
# Check user goals were met
dialog_result = evaluate_dialog(user_sim.goal)

print(user_sim.goal.get_goal())

if dialog_result:
    print("Dialog was successful")
else:
    print("Dialog Failed")

{'inform_slots': {'pricerange': 'moderate', 'area': 'north'}, 'request_slots': {'name': 'UNK', 'phone': 'UNK'}}
1 :  Agent -> greetings None
Agent :  None
1 :  User -> inform {'area': 'north'}
User :  Looking for a food in north.
2 :  Agent -> request {'cuisine'}
Agent :  None
2 :  User -> inform {'no_pref': 'UNK'}
User :  whatever
3 :  Agent -> request {'cuisine'}
Agent :  None
3 :  User -> inform {'no_pref': 'UNK'}
User :  whatever
4 :  Agent -> request {'cuisine'}
Agent :  None
4 :  User -> inform {'no_pref': 'UNK'}
User :  whatever
5 :  Agent -> request {'cuisine'}
Agent :  None
5 :  User -> inform {'no_pref': 'UNK'}
User :  whatever
6 :  Agent -> request {'cuisine'}
Agent :  None
6 :  User -> inform {'no_pref': 'UNK'}
User :  I don't care
7 :  Agent -> request {'cuisine'}
Agent :  None
7 :  User -> inform {'no_pref': 'UNK'}
User :  whatever
8 :  Agent -> request {'cuisine'}
Agent :  None
8 :  User -> inform {'no_pref': 'UNK'}
User :  up for anything.
{'inform_slots': {'pricerange'

In [20]:
DialogStatus.FINISHED

<DialogStatus.FINISHED: 1>

In [ ]:
user_sim.reset()

In [ ]:
user_sim.generate_goal("template")
user_sim.goal.get_goal()

In [ ]:
f = {}
f.("test")